In [ ]:
flex_subtitle = "Built using jupyter-flex"
flex_source_link = "https://github.com/josefmtd/heroku-gee-fwi/blob/main/FWI.ipynb"
flex_include_source = True

In [ ]:
import ee
import geemap.foliumap as geemap
from datetime import date, timedelta

# Interactive Python Widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

## Section

In [ ]:
obs_label = widgets.Label(value='Date:')
obs_var = widgets.DatePicker(
    value=date.today() - timedelta(days=1)
)
widgets.VBox([obs_label, obs_var])

## Column

In [ ]:
out = widgets.Output()

In [ ]:
def on_value_change(change):
    obs = obs_var.value

    with out:
        Map = geemap.Map(center=(-3.1, 104.1), zoom=8,
            add_google_map=False)

        url = f'gs://fwi-bicubic-outputs/ID-SS_{obs.isoformat()}_FWI.tif'
        fwi_data = ee.Image.loadGeoTIFF(url)
        palette = ['blue', 'green', 'yellow', 'red']

        fwi_vis_params = {
            'min': 0.0, 'max': 13.0, 'palette': palette
        }

        vmin = fwi_vis_params['min']
        vmax = fwi_vis_params['max']

        sumatera_selatan = ee.FeatureCollection('FAO/GAUL/2015/level2') \
            .filter(ee.Filter.eq('ADM1_NAME', 'Sumatera Selatan'))

        Map.addLayer(fwi_data.select('FWI'), fwi_vis_params, 'Fire Weather Index')
        Map.addLayer(sumatera_selatan, {'fillColor' : 'ffffff'}, 'South Sumatra')
        Map.add_colorbar(colors=palette, vmin=vmin, vmax=vmax)
        clear_output(wait=True)
        display(Map)

obs_var.observe(on_value_change, names="value")

In [ ]:
on_value_change(None)
out